In [1]:
import json
import itertools
from fractions import Fraction
import numpy as np
import pandas

#GUI elements
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

In [2]:
%run logic.ipynb
%run participant.ipynb


----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK


In [3]:
#Example JSON
root_question = {'qid':0,
'question': 'Q1',
'answers': ['A','B','C'],
'probability': ['1/3','1/3','1/3']}

follow_up_1 = {'qid':1,
'question': 'Follow-up...',
'answers': ['1','2','3','4','5'],
'probability': ['1/5','1/5','1/5','1/5','1/5']}

paths = [[0,'C',1]]

json_poll = {
    'roots': [root_question],
    'children': [follow_up_1],
    'paths':paths,
    'truth':['1/4']
}

In [4]:
answer_dist = {}
poll_name = ''

In [5]:
def loadJSON(change):
    global json_poll
    global poll_name
    poll_name = file_selection.value 
    json_poll = getJSON(poll_name)

def getJSON(name):
    # Read JSON from files
    json_file = open(name, 'r')
    json_content = json_file.read()
    return json.loads(json_content)

In [6]:
def update_dist(change):
    
    current_accordion = dist_accordion.children[dist_accordion.selected_index]
    dropdown = current_accordion.children[1]
    active_sliders = current_accordion.children[2].children
    
    if dropdown.value != 'Custom':
        shape_slider = current_accordion.children[3]

        # Draw values from dist
        values = distMap[dropdown.value](len(active_sliders),shape_slider.value)

        for slider in active_sliders:

            index = active_sliders.index(slider)
            slider.value = np.round(values[index], decimals=sliders_precision)
            
     
    #Save values
    for slider in active_sliders:
        index = active_sliders.index(slider)
        answer_dist[dist_accordion.selected_index][index][dropdown.value] = slider.value

        

In [7]:
def dropdown_action(change):
    current_accordion = dist_accordion.children[dist_accordion.selected_index]
    dropdown = current_accordion.children[1]
    shape_slider = current_accordion.children[3]
    shape_slider.hold_trait_notifications()

    # Update the shape slider according to the chosen distribution
    if(dropdown.value is 'Uniform' or dropdown.value is 'Custom'):
        shape_slider.disabled=True
        shape_slider.layout.visibility='hidden'
    else:

        if(dropdown.value is 'Geom'):
            shape_slider.step = 0.01
            shape_slider.min = 0. + shape_slider.step
            shape_slider.max = 1.
            shape_slider.value = 0.5
        elif(dropdown.value is 'Logser'):
            shape_slider.step = 0.01
            shape_slider.min = 0. + shape_slider.step
            shape_slider.max = 1. - shape_slider.step
            shape_slider.value = 0.6
        elif(dropdown.value is 'Planck'):
            shape_slider.step = 0.01
            shape_slider.min = 0. + shape_slider.step
            shape_slider.max = 1.
            shape_slider.value = 0.51
        elif(dropdown.value is 'Zipf'):
            shape_slider.max = 10.
            shape_slider.step = 0.01
            shape_slider.min = 1. + shape_slider.step
            shape_slider.value = 1.5

        shape_slider.disabled=False
        shape_slider.layout.visibility='visible'
        
    #Reset values
    active_sliders = current_accordion.children[2].children   
     
    for slider in active_sliders:
        index = active_sliders.index(slider)
        
        val = dropdown.value
        if val not in answer_dist[dist_accordion.selected_index][index].keys():
            update_dist(None)
        slider.value = answer_dist[dist_accordion.selected_index][index][val]

In [8]:
%run gui_components.ipynb

<IPython.core.display.Javascript object>

In [9]:
display(widgets.HTML('<center><h1> Choose Parameters'))
load_button.on_click(loadJSON)
display(start_tab)

HTML(value=u'<center><h1> Choose Parameters')

VGFiKGNoaWxkcmVuPShIQm94KGNoaWxkcmVuPShWQm94KGNoaWxkcmVuPShMYWJlbCh2YWx1ZT11J0lucHV0IEpTT046JyksIFNlbGVjdChvcHRpb25zPSgnYmxvb2QuanNvbicsKSwgdmFsdWXigKY=


In [15]:
# Distribution sliders per question (non-flattened)
children = []

subtrees = pollToSubtrees(json_poll)
for question in subtrees:
    for subquestion in subtrees[question]:
        dist_accordion.set_title(subquestion['qid'], 'Question: '+subquestion['question'])
        html = widgets.HTML('<h3>Choose weight distribution...</h3>')
        to_add = []
        answer_dist[subquestion['qid']] = {}
        
        count = 0
        for answer in subquestion['answers']:
            probability = subquestion['probability'][count]
            to_add.append(widgets.FloatSlider(
                value=probability,
                min=0,
                max=1.0,
                step=0.01,
                description=answer,
                disabled=False,
                continuous_update=False,
                orientation='vertical',
                readout=True,
                readout_format='.1f',
            ))
            answer_dist[subquestion['qid']][count] = {'Custom': probability}
            count +=1
        
        #Components
        _shape_slider = widgets.FloatSlider(min=0., max=5., step=0.1, value=0., continuous_update=False, description='Shape', disabled=True) 
        _dropdown = widgets.Dropdown(options=dists, value='Custom', description='Distribution:', disabled=False)
        _button = widgets.Button(description='Update weights', icon='fa-bar-chart', button_style='success')
        
        #Actions
        _button.on_click(update_dist)
        _dropdown.observe(dropdown_action)
        
        contents =[html, _dropdown, widgets.HBox(to_add),_shape_slider, _button]
        children.append( widgets.VBox(contents))
        
dist_accordion.children = children
dist_box = widgets.VBox([dist_accordion])

In [16]:
display(widgets.HTML('<center><h1> Set <i>True</i> Answer Distribution</h1></center>'))
display(dist_accordion)
dropdown_action(None)

HTML(value=u'<center><h1> Set <i>True</i> Answer Distribution</h1></center>')

QWNjb3JkaW9uKGNoaWxkcmVuPShWQm94KGNoaWxkcmVuPShIVE1MKHZhbHVlPXUnPGgzPkNob29zZSB3ZWlnaHQgZGlzdHJpYnV0aW9uLi4uPC9oMz4nKSwgRHJvcGRvd24oZGVzY3JpcHRpb27igKY=


In [17]:
matrices = pollToMatrix(json_poll)
epsilons = pollToEpsilon(json_poll)
ps = pollToPaths(json_poll)

In [18]:
selected = {}
for question in subtrees:
    selected[question] = {}
    for subquestion in subtrees[question]:
        selected[question][subquestion['qid']] = []
        current_box = dist_accordion.children[subquestion['qid']].children
        current_dropdown = current_box[1]
        current_sliders = current_box[2].children
  
        #NORMALIZE SLIDER VALUES BEFORE MOVING ON
        normalize_by = 0
        for slider in current_sliders:
            index = current_sliders.index(slider)
            normalize_by+= answer_dist[subquestion['qid']][index][current_dropdown.value]

        for slider in current_sliders:
            index = current_sliders.index(slider)
            value = answer_dist[subquestion['qid']][index][current_dropdown.value]/normalize_by
            selected[question][subquestion['qid']].append(value)
            


# Find matching path
# {qid: (Path, Fraction)}

In [19]:
# Turn slides to JSON, then parse to find path-weight pairs

#FIXME: Don't overwrite actual json_poll, do deep copy!
 
dist_poll = getJSON(poll_name)

for question in subtrees:
    root = filter(lambda x: x['qid']==question, dist_poll['roots'])[0]
    root['probability'] = selected[question][question]
    
    for subquestion in subtrees[question]:
        # Roots have already been added, will also return empty list so ignore
        question_struct = filter(lambda x: x['qid']==subquestion['qid'], dist_poll['children'])
        if len(question_struct) > 0:
            question_struct[0]['probability'] = selected[question][subquestion['qid']]
            
            
weights = pollToPathWeights(dist_poll)

In [26]:
ans_dist_zip = {}
for question in ps.keys():
    paths = ps[question]
    weight = weights[question]
    zipped = []
    for path in paths:
        zipped.append((path, weights[question][pathToKey(path)]))

    ans_dist_zip[question] = zipped

In [27]:
# Add widget

In [111]:
participants = []
n = 10000
participant_budget = 40
runs = 10
relative_error = {}
absolute_error = {}

#TODO: Loop x times for confidence
for run in range(0, runs):
    raw_responses = []
    real_answers = []

    #Initialize partitipants and run RR
    for i in range(0, n):
        new_participant = Participant(participant_budget, json_poll)
        new_participant.setMatrices(matrices)
        new_participant.setEpsilons(epsilons)
        new_participant.createAnswer(ans_dist_zip)

        real_answers.append(dict(new_participant.getAnswer())) #Beware: deep copy required!
        raw_responses.append(new_participant.randomizedResponse())

        participants.append(new_participant)

    # Parse responses
    lists_responses = {}
    lists_answer = {}
    for question in subtrees.keys():
        lists_responses[question] = map(lambda x: pathToKey(x[question]), raw_responses)
        lists_answer[question] = map(lambda x: pathToKey(x[question]), real_answers)

    response_frame = pd.DataFrame(lists_responses)
    response_frequency = {}
    answer_frame = pd.DataFrame(lists_answer)
    answer_frequency = {}

    #Count occurences
    for question in subtrees.keys():
        response_frequency[question] = {}
        answer_frequency[question] = {}

        to_match = pd.unique(response_frame[question])
        for match in to_match:
            response_frequency[question][match] = len(response_frame[response_frame[question]==match])
            answer_frequency[question][match] = len(answer_frame[answer_frame[question]==match])

    #Filter with Bayes' theorem
    for question in response_frequency.keys():
        absolute_error[question] = {}
        relative_error[question] = {}
        #P(True)
        p_true = Fraction(json_poll['truth'][question])

        #Bayes' theorem: p(A|True) = p(True|A)*p(A) / p(True)
        for alternative in response_frequency[question].keys():
            transition = (alternative, alternative)
            #p(True|A), Probability of not changing answer
            p_true_given_a = matrices[question][transition]
            #p(A), Actual responses
            p_a = float(response_frequency[question][alternative])/len(participants) #Convert to %

            #p(A|True) = p(True|A)*p(A) / p(True)
            p_a_given_true = (p_true_given_a*p_a)/p_true
            
            real_percentage = float(answer_frequency[question][alternative])/len(participants)

            abs_diff = abs(p_a_given_true-real_percentage)
            rel_diff = 100
            if real_percentage != 0:
                rel_diff = abs_diff/real_percentage
            
            absolute_error[question][alternative] = abs_diff
            relative_error[question][alternative] = rel_diff


In [113]:
pd.DataFrame(absolute_error)

,0
A+,0.011280
O+,0.010857
OtherA-,0.000016
OtherAB+,0.000831
OtherB+,0.000515
OtherO-,0.000688
OtherOtherAB-,0.000021
OtherOtherB-,0.000115


In [112]:
pd.DataFrame(relative_error)

,0
A+,0.339350
O+,0.321108
OtherA-,0.002247
OtherAB+,0.133440
OtherB+,0.074374
OtherO-,0.117808
OtherOtherAB-,0.006176
OtherOtherB-,0.033333


In [85]:

#TODO: display graphs